In [1]:
# from desktop w/ gpu
import torch
import numpy as np
import cv2
import time
import os
from enum import Enum
from multipledispatch import dispatch
import mrs3 as mr
import interpolation as inter

%load_ext autoreload
%autoreload 2

lenna_path = 'Lenna_(test_image).png'

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(device = 0))
print(torch.cuda.device_count())
print(torch.backends.cudnn.version())



True
NVIDIA GeForce RTX 3060
1
91000


여러 파일을 하나의 파일(pkg)로 묶어서 저장, 읽기 및 복원

In [3]:
mr.compress_img_mult_tgs('Downloads/500x500.png', 'testf', scaler=4, roi_mode=mr.ROI_POLYGON)

original file: 325420
downscaled filesize: 31065
roi filesize: 34880
roi bin filesize: 764
config filesize: 149
compression ratio: 0.20545141663081556


In [4]:
mr.restore_img_mult_tgs('testf', mr.EDSR, 'testf')

1.078644037246704 sec taken


compress 이후에 packaging 처리

In [5]:
import os
import struct

def pack_files(output_file: str, input_files: list):
    """
    여러 파일을 하나의 사용자 정의 패키지(.pkg)로 묶습니다.
    
    :param output_file: 출력 파일 경로 (예: 'output.pkg')
    :param input_files: 패키징할 파일 목록 (예: ['image.png', 'config.ini'])
    """
    with open(output_file, 'wb') as f_out:
        # 파일 개수 기록 (4바이트)
        f_out.write(struct.pack('I', len(input_files)))
        
        for file_path in input_files:
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"파일을 찾을 수 없음: {file_path}")
            
            file_name = os.path.basename(file_path)
            with open(file_path, 'rb') as f_in:
                file_data = f_in.read()
            
            # 파일명 길이 및 데이터 기록
            encoded_name = file_name.encode('utf-8')
            f_out.write(struct.pack('I', len(encoded_name)))  # 파일명 길이 (4바이트)
            f_out.write(encoded_name)                         # 파일명 데이터
            
            # 파일 내용 기록
            f_out.write(struct.pack('I', len(file_data)))     # 파일 크기 (4바이트)
            f_out.write(file_data)                            # 파일 내용

def unpack_files(input_file: str, output_dir: str):
    """
    패키지 파일(.pkg)에서 원본 파일을 추출합니다.
    
    :param input_file: 입력 파일 경로 (예: 'output.pkg')
    :param output_dir: 출력 디렉토리 경로 (예: 'unpacked')
    """
    os.makedirs(output_dir, exist_ok=True)
    
    with open(input_file, 'rb') as f_in:
        # 파일 개수 읽기
        num_files = struct.unpack('I', f_in.read(4))[0]
        
        for _ in range(num_files):
            # 파일명 추출
            name_len = struct.unpack('I', f_in.read(4))[0]
            file_name = f_in.read(name_len).decode('utf-8')
            
            # 파일 내용 추출
            data_len = struct.unpack('I', f_in.read(4))[0]
            file_data = f_in.read(data_len)
            
            # 파일 저장
            output_path = os.path.join(output_dir, file_name)
            with open(output_path, 'wb') as f_out:
                f_out.write(file_data)

# 사용 예시 ---------------------------------------------------

# 파일 패키징
pack_files(
    output_file="combined.pkg",
    input_files=["/home/primarina314/Projects/MRS3/testf/bin0.png", "/home/primarina314/Projects/MRS3/testf/bin1.png", "/home/primarina314/Projects/MRS3/testf/roi0.png", "/home/primarina314/Projects/MRS3/testf/roi1.png", "/home/primarina314/Projects/MRS3/testf/downscaled.png", "/home/primarina314/Projects/MRS3/testf/config.ini"]
)

# 파일 복원
unpack_files(
    input_file="combined.pkg",
    output_dir="restored_files"
)
